In [1]:
import json
import os
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM, BruteForce, MAPOCAM
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import GeneralClassifier_Shap, GeneralClassifier
from cfmining.action_set import ActionSet
from cfmining.utils import get_data_model

import dice_ml
from nice import NICE


%load_ext autoreload
%autoreload 2

In [2]:
SEED = 0

## Helper

In [3]:
class DiCE_wrapper:
    def __init__(self, dice_data, dice_model, total_CFs, mutable_features, sparsity_weight = 0.2):
        self.total_CFs = total_CFs
        self.sparsity_weight = sparsity_weight
        self.mutable_features = mutable_features
        self.exp = dice_ml.Dice(dice_data, dice_model)


    def fit(self, individual):
        dice_exp = self.exp.generate_counterfactuals(
            individual,
            total_CFs = self.total_CFs,
            desired_class = "opposite",
            sparsity_weight = self.sparsity_weight,
            features_to_vary= self.mutable_features,
        )
        solutions  = json.loads(dice_exp.to_json())["cfs_list"][0]
        self.solutions = [solution[:-1] for solution in solutions]
        return self


def get_dice_results(
    method,
    model,
    individuals,
    output_file = None,
):
    results = []
    
    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[[i]]
        model.clear_cache()
        start = time.time()
        method.fit(individual)
        end = time.time()

        results.append({
            "individual" : individual.values.tolist()[0],
            "prob" : model.predict_proba(individual.values[0]),
            "time" : end - start,
            "n_solutions" : len(method.solutions),
            "solutions" : method.solutions,
        })
    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results

In [4]:
def get_nice_results(
    method,
    model,
    individuals,
    output_file=None,
   
):
    

    results = []

    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[[i]]
        start = time.time()

        # Generate counterfactuals using NICE
        cf = method.explain(individual.values).tolist()

        end = time.time()

        results.append({
            "individual": individual.values.tolist()[0],
            "prob": model.predict_proba(individual.values[0]),
            "time": end - start,
            "n_solutions": len(cf),
            "solutions": cf,
        })

    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results


In [5]:
class MAPOCAM_wrapper:
    def __init__(self, action_set, model, criteria, max_changes):
        self.action_set = action_set
        self.model = model
        if criteria == "percentile":
            perc_calc = PercentileCalculator(action_set = action_set)
            self.compare = lambda ind : PercentileCriterion(ind, perc_calc)
        elif criteria == "percentile_changes":
            perc_calc = PercentileCalculator(action_set = action_set)
            self.compare = lambda ind : PercentileChangesCriterion(ind, perc_calc)
        elif criteria == "nom_dom":
            self.compare = lambda ind : NonDomCriterion(ind)
            
        self.max_changes = max_changes

    def fit(self, individual):
        m = MAPOCAM(
            self.action_set,
            individual,
            self.model, 
            max_changes = self.max_changes,
            compare = self.compare(individual)
        )
        m.fit()
        self.solutions = m.solutions
        return self


In [6]:
class BruteForce_wrapper:
    def __init__(self, action_set, model, criteria, max_changes):
        self.action_set = action_set
        self.model = model
        if criteria == "percentile":
            perc_calc = PercentileCalculator(action_set = action_set)
            self.compare = lambda ind : PercentileCriterion(ind, perc_calc)
        elif criteria == "percentile_changes":
            perc_calc = PercentileCalculator(action_set = action_set)
            self.compare = lambda ind : PercentileChangesCriterion(ind, perc_calc)
        elif criteria == "nom_dom":
            self.compare = lambda ind : NonDomCriterion(ind)
            
        self.max_changes = max_changes

    def fit(self, individual):
        m = BruteForce(
            self.action_set,
            individual,
            self.model, 
            max_changes = self.max_changes,
            compare = self.compare(individual)
        )
        m.fit()
        self.solutions = m.solutions
        return self

In [7]:
def get_mapofcem_results(
        method,
        individuals, 
        model, 
        output_file = None,
    ):
    results = []

    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[i]
        model.clear_cache()
        start = time.time()
        method.fit(individual.values)
        end = time.time()

        solutions = method.solutions
        solutions = [s.tolist() for s in solutions]
        
        results.append({
            "individual" : individual.values.tolist(),
            "prob" : model.predict_proba(individual.values),
            "time" : end - start,
            "n_solutions" : len(method.solutions),
            "solutions" : solutions,
        })

        if output_file is not None:
            pd.DataFrame(results).to_csv(output_file, index=False)

        

    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results

In [8]:
def run_all_mapofcem_variations(
    dataset_name,
    action_set,
    individuals,
    model_shap,
    outlier_percentile,
):
    # MAPOFCEM

    ## MPC

    method = MAPOFCEM(
        action_set,
        model_shap,
        compare = "percentile",
        estimate_prob_max=True,
        estimate_outlier=False,
        max_changes = 3,
        outlier_percentile = outlier_percentile
    )
    

    get_mapofcem_results(
        method=method,
        individuals=individuals,
        model=model_shap,
        output_file=f"../results/{dataset_name}/mapofcem_percentile_outlier_{outlier_percentile}.csv"
    )


    method = MAPOFCEM(
        action_set,
        model_shap,
        compare = "percentile",
        estimate_prob_max=True,
        estimate_outlier=True,
        max_changes = 3,
        outlier_percentile = outlier_percentile
    )

    get_mapofcem_results(
        method=method,
        individuals=individuals,
        model=model_shap,
        output_file=f"../results/{dataset_name}/mapofcem_v2_percentile_outlier_{outlier_percentile}.csv"
    )


    return

## German

In [9]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("german", "LGBMClassifier")
individuals = individuals.sample(n = 50, random_state=SEED)

In [10]:
not_mutable_features = ['Age', 'OwnsHouse', 'isMale', 'JobClassIsSkilled', 'Single', 'ForeignWorker', 'RentsHouse']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ['Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome','YearsAtCurrentHome']
categoric_features = [col for col in X_train.columns if col not in continuous_features]

action_set = ActionSet(X = X_train, default_step_size = 0.05, mutable_features = mutable_features)

In [11]:
model_shap = GeneralClassifier_Shap(model, outlier_detection, X_train, shap_explainer="tree", threshold = 0.5)

### MAPOFCEM

In [12]:
for outlier_percentile in [0.005, 0.01, 0.05, 0.075, 0.1]:
    run_all_mapofcem_variations(
        dataset_name="german",
        action_set=action_set,
        individuals=individuals,
        model_shap=model_shap,
        outlier_percentile=outlier_percentile,
    )

100%|███████████████████████████████████████████| 50/50 [01:13<00:00,  1.46s/it]


## Taiwan

In [13]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("taiwan")
#convert bools to int
X_train = X_train.astype(int)
individuals = individuals.astype(int)
individuals = individuals.sample(n = 50, random_state=SEED)

In [14]:
not_mutable_features = ['Single', 'Age_in_25_to_40', 'Married', 'Age_lt_25', 'Age_in_40_to_59', 'Age_geq_60', 'EducationLevel']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ["LIMIT_BAL", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                       "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1",
                       "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5",
                       "PAY_AMT6", "MaxBillAmountOverLast6Months", "MaxPaymentAmountOverLast6Months",
                       "MostRecentBillAmount", "MostRecentPaymentAmount", "MostRecentPaymentAmount", "TotalMonthsOverdue"]
categoric_features = [col for col in X_train.columns if col not in continuous_features]
action_set = ActionSet(X = X_train, default_step_size = 0.1, mutable_features = mutable_features)

### MAPOFCEM

In [15]:
model_shap = GeneralClassifier_Shap(model, outlier_detection, X_train, shap_explainer="tree", threshold = 0.5)

In [16]:
for outlier_percentile in [0.005, 0.01, 0.05, 0.075, 0.1]:
    run_all_mapofcem_variations(
        dataset_name="taiwan",
        action_set=action_set,
        individuals=individuals,
        model_shap=model_shap,
        outlier_percentile=outlier_percentile,
    )

100%|███████████████████████████████████████████| 50/50 [26:18<00:00, 31.56s/it]
